In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Load data from Excel (replace with the correct file path)
df = pd.read_excel('../data.xlsx', sheet_name='Rekapan')

In [ ]:
# Display the first and last few rows of the dataset
print(df.head())
print(df.tail())

In [ ]:
# Display basic descriptive statistics
print(df.describe())

# Check for missing values
print(df.isnull().sum())

# Visualize data distribution
df.hist(bins=50, figsize=(20, 15))
plt.show()

In [ ]:
# Filter relevant columns
data_filtered = df[['Tanggal', 'SO', 'TERKIRIM', 'Harga Komoditas Bijih Besi', 'Indeks Produksi Dalam Negeri', 'Data Inflasi', 'Kurs']]

# Rename columns for easier handling
data_filtered.columns = ['Tanggal', 'SO', 'Terkirim', 'Harga Komoditas', 'Indeks Produksi', 'Data Inflasi', 'Kurs']

# Convert 'Tanggal' column to datetime
data_filtered['Tanggal'] = pd.to_datetime(data_filtered['Tanggal'])

# Convert 'Indeks Produksi' to numeric (coercing errors)
data_filtered['Indeks Produksi'] = pd.to_numeric(data_filtered['Indeks Produksi'], errors='coerce')

# Create 'Year' column to group by year
data_filtered['Year'] = data_filtered['Tanggal'].dt.year

# Yearly Summary by 'Year'
yearly_summary = data_filtered.groupby('Year').agg({
    'SO': 'sum',
    # Uncomment other columns as needed
    # 'Terkirim': 'sum',
    # 'Harga Komoditas': 'mean',
    # 'Indeks Produksi': 'mean',
    # 'Data Inflasi': 'mean',
    # 'Kurs': 'mean'
}).reset_index()

# Display yearly summary
print(yearly_summary)

In [ ]:
# Set alpha value for Double Exponential Smoothing (DES)
alpha = 0.1

# Initialize columns for DES and error metrics
yearly_summary['Single'] = np.nan
yearly_summary['Double'] = np.nan
yearly_summary['At'] = np.nan
yearly_summary['Bt'] = np.nan
yearly_summary['DES Forecast'] = np.nan
yearly_summary['Error'] = np.nan
yearly_summary['MAD'] = np.nan
yearly_summary['MSE'] = np.nan
yearly_summary['MAPE'] = np.nan

# Set initial Single, Double, At, and Bt values based on the first observation
yearly_summary.loc[0, 'Single'] = yearly_summary.loc[0, 'SO']
yearly_summary.loc[0, 'Double'] = yearly_summary.loc[0, 'SO']
yearly_summary.loc[0, 'At'] = (2 * yearly_summary.loc[0, 'Single']) - yearly_summary.loc[0, 'Double']
yearly_summary.loc[0, 'Bt'] = 0  # Set initial trend (Bt) to 0

# Apply DES formula and calculate error metrics for each subsequent row
for i in range(1, len(yearly_summary)):
    # Single Exponential smoothing (S't)
    yearly_summary.loc[i, 'Single'] = (alpha * yearly_summary.loc[i, 'SO']) + ((1 - alpha) * yearly_summary.loc[i-1, 'Single'])
    
    # Double Exponential smoothing (S''t)
    yearly_summary.loc[i, 'Double'] = (alpha * yearly_summary.loc[i, 'Single']) + ((1 - alpha) * yearly_summary.loc[i-1, 'Double'])
    
    # At (level)
    yearly_summary.loc[i, 'At'] = (2 * yearly_summary.loc[i, 'Single']) - yearly_summary.loc[i, 'Double']
    
    # Bt (trend)
    yearly_summary.loc[i, 'Bt'] = (alpha / (1 - alpha)) * (yearly_summary.loc[i, 'Single'] - yearly_summary.loc[i, 'Double'])
    
    # DES Forecast for the next period
    if i > 0:  # Forecast from the second row onward
        yearly_summary.loc[i, 'DES Forecast'] = yearly_summary.loc[i-1, 'At'] + yearly_summary.loc[i-1, 'Bt']
    
    # Error (actual - forecast)
    yearly_summary.loc[i, 'Error'] = yearly_summary.loc[i, 'SO'] - yearly_summary.loc[i, 'DES Forecast']
    
    # MAD (Mean Absolute Deviation)
    yearly_summary.loc[i, 'MAD'] = abs(yearly_summary.loc[i, 'Error'])
    
    # MSE (Mean Squared Error)
    yearly_summary.loc[i, 'MSE'] = yearly_summary.loc[i, 'Error'] ** 2
    
    # MAPE (Mean Absolute Percentage Error)
    if yearly_summary.loc[i, 'SO'] != 0:
        yearly_summary.loc[i, 'MAPE'] = (abs(yearly_summary.loc[i, 'Error']) / yearly_summary.loc[i, 'SO']) * 100
    else:
        yearly_summary.loc[i, 'MAPE'] = np.nan

# Show the result with DES forecast and error metrics
print(yearly_summary)

In [ ]:
# Forecast for future years (6 years ahead)
years_to_forecast = 6

# Get the last year from the original data
last_year = yearly_summary['Year'].max()

# Generate dummy years starting from the next year after the last year in the original data
dummy_years = pd.DataFrame({
    'Year': range(last_year + 1, last_year + 1 + years_to_forecast),
    'SO': np.zeros(years_to_forecast)  # Set SO to 0 for future years
})

# Combine original data with the dummy future data
extended_data = pd.concat([yearly_summary[['Year', 'SO']], dummy_years], ignore_index=True)

# Initialize columns for DES forecast in the extended data
extended_data['Single'] = np.nan
extended_data['Double'] = np.nan
extended_data['At'] = np.nan
extended_data['Bt'] = np.nan
extended_data['DES Forecast'] = np.nan
extended_data['Error'] = np.nan
extended_data['MAD'] = np.nan
extended_data['MSE'] = np.nan
extended_data['MAPE'] = np.nan

# Inisialisasi nilai awal untuk peramalan berdasarkan nilai historis pertama
extended_data.loc[0, 'Single'] = extended_data.loc[0, 'SO']
extended_data.loc[0, 'Double'] = extended_data.loc[0, 'SO']
extended_data.loc[0, 'At'] = (2 * extended_data.loc[0, 'Single']) - extended_data.loc[0, 'Double']
extended_data.loc[0, 'Bt'] = 0  # Set initial trend to 0

# Apply DES formula to historical data and future periods
for i in range(1, len(extended_data)):
    # Use the DES formula for historical and future periods
    extended_data.loc[i, 'Single'] = (alpha * extended_data.loc[i, 'SO']) + ((1 - alpha) * extended_data.loc[i-1, 'Single'])
    extended_data.loc[i, 'Double'] = (alpha * extended_data.loc[i, 'Single']) + ((1 - alpha) * extended_data.loc[i-1, 'Double'])
    extended_data.loc[i, 'At'] = (2 * extended_data.loc[i, 'Single']) - extended_data.loc[i, 'Double']
    extended_data.loc[i, 'Bt'] = (alpha / (1 - alpha)) * (extended_data.loc[i, 'Single'] - extended_data.loc[i, 'Double'])
    extended_data.loc[i, 'DES Forecast'] = extended_data.loc[i-1, 'At'] + extended_data.loc[i-1, 'Bt']
    
    # Error
    extended_data.loc[i, 'Error'] = extended_data.loc[i, 'SO'] - extended_data.loc[i, 'DES Forecast']
    
    # MAD (Mean Absolute Deviation)
    extended_data.loc[i, 'MAD'] = abs(extended_data.loc[i, 'Error'])
    
    # MSE (Mean Squared Error)
    extended_data.loc[i, 'MSE'] = extended_data.loc[i, 'Error'] ** 2
    
    # MAPE (Mean Absolute Percentage Error)
    if extended_data.loc[i, 'SO'] != 0:
        extended_data.loc[i, 'MAPE'] = (abs(extended_data.loc[i, 'Error']) / extended_data.loc[i, 'SO']) * 100
    else:
        extended_data.loc[i, 'MAPE'] = np.nan

# Display the extended data with future forecasts
print(extended_data)


In [ ]:
# Plot Actual SO vs DES Forecast, including future years
plt.figure(figsize=(14, 6))
plt.plot(extended_data['Year'], extended_data['SO'], label='Actual SO', marker='o')
plt.plot(extended_data['Year'], extended_data['DES Forecast'], label='DES Forecast', marker='x')
plt.xlabel('Year')
plt.ylabel('SO')
plt.xticks(rotation=45)
plt.title('Actual SO vs DES Forecast (Including Future Years)')
plt.legend()
plt.tight_layout()
plt.show()